# 9장 Recommendation Engines Using MapReduce

## 추천 시스템이란 

아마존에서는 "Frequently Bought Together"(아이템기반 추천) 과 "Customers Who Bought This Item Also Bought"(사용자기반 추천)의 여러 가지 추천 아이템을 보여줌.

https://www.amazon.com/Data-Algorithms-Recipes-Scaling-Hadoop/dp/1491906189/ref=sr_1_1?ie=UTF8&qid=1477648946&sr=8-1&keywords=data+algorithms

![](spark09_04.jpg)


추천 엔진과 시스템은 사용자에게 아래와 같은 사용자 경험을 증진시켜줌.
- 사용자에게 찾는 정보를 조언해줌.
- 아이템(상품)을 검색과 네비게이션을 시간을 줄어줌.
- 사이트에 자주 방문하도록 만족도을 올려줌.

추천 엔진과 시스템의 목적
- 사용자가 아직 사지 않은 아이템을 추천
- 사용자가 생각지도 못한 영화나 책을 추천
- 사용자가 방문하지 않은 음식점이나 장소 추천

## Friendship connection

![](spark09_01.jpg)

### Input Data

![](spark09_02.jpg)

- friends2.txt
```
1 2,3,4,5,6,7,8
2 1,3,4,5,7
3 1,2
4 1,2,6
5 1,2
6 1,4
7 1,2
8 1
```

### Output 

- USER :  F(M: [I1, I2, I3, ...]), ...
    - F : USER에게 친구로 추천하는 사람의 ID
    - M : 같이 친구의 명수 
    - I1, I2, I3,  : 같이 친구인 사람의 ID

```
4: 3 (2: [1, 2]),5 (2: [1, 2]),7 (2: [1, 2]),8 (1: [1]),
2: 6 (2: [1, 4]),8 (1: [1]),
6: 2 (2: [1, 4]),3 (1: [1]),5 (1: [1]),7 (1: [1]),8 (1: [1]),
8: 2 (1: [1]),3 (1: [1]),4 (1: [1]),5 (1: [1]),6 (1: [1]),7 (1: [1]),
3: 4 (2: [1, 2]),5 (2: [1, 2]),6 (1: [1]),7 (2: [1, 2]),8 (1: [1]),
1:
7: 3 (2: [1, 2]),4 (2: [1, 2]),5 (2: [1, 2]),6 (1: [1]),8 (1: [1]),
5: 3 (2: [1, 2]),4 (2: [1, 2]),6 (1: [1]),7 (2: [1, 2]),8 (1: [1]),
```

## Spark Implementation

![](spark09_03.jpg)

### Step 3: Create a Spark context object

In [2]:
from pyspark import SparkContext
sc = SparkContext() 
sc

### Step 4: Read the HDFS input file and create an RDD

In [36]:
records = sc.textFile("friends2.txt", 1);

In [38]:
for t in records.collect():
    print "debug0 record:", t

debug0 record: 1	2,3,4,5,6,7,8
debug0 record: 2	1,3,4,5,7
debug0 record: 3	1,2
debug0 record: 4	1,2,6
debug0 record: 5	1,2
debug0 record: 6	1,4
debug0 record: 7	1,2
debug0 record: 8	1


### Step 5: Implement the map() function

In [39]:
def make_pairs( record ) :
    # // record=<person><TAB><friend1><,><friend2><,><friend3><,>...
    tokens = record.split("\t")
    person = long( tokens[0] )
    friendsAsString = tokens[1]
    friendsTokenized = friendsAsString.split(",");
    
    friends = []  ## LIST형
    mapperOutput = [] ## LIST형
    for friendAsString in  friendsTokenized :
        toUser = long( friendAsString )
        friends.append( toUser  ) 
        directFriend = ( toUser, -1L )  # 튜플형
        mapperOutput.append( ( person, directFriend )  )
        
    for i  in range( len(friends) )  :
        for j in range( i+1,  len(friends) )  :
            possibleFriend1 = ( friends[j], person )
            mapperOutput.append( (friends[i], possibleFriend1)  ) 
            
            possibleFriend2 = ( friends[i], person )
            mapperOutput.append( (friends[j], possibleFriend2) ) 
            
    return mapperOutput

In [40]:
pairs = records.flatMap( make_pairs  )

In [41]:
debug2 = pairs.collect()
for t2 in debug2 :
    print "debug1 key={}\t value={}".format( t2[0],  t2[1] ) ; 

debug1 key=1	 value=(2L, -1L)
debug1 key=1	 value=(3L, -1L)
debug1 key=1	 value=(4L, -1L)
debug1 key=1	 value=(5L, -1L)
debug1 key=1	 value=(6L, -1L)
debug1 key=1	 value=(7L, -1L)
debug1 key=1	 value=(8L, -1L)
debug1 key=2	 value=(3L, 1L)
debug1 key=3	 value=(2L, 1L)
debug1 key=2	 value=(4L, 1L)
debug1 key=4	 value=(2L, 1L)
debug1 key=2	 value=(5L, 1L)
debug1 key=5	 value=(2L, 1L)
debug1 key=2	 value=(6L, 1L)
debug1 key=6	 value=(2L, 1L)
debug1 key=2	 value=(7L, 1L)
debug1 key=7	 value=(2L, 1L)
debug1 key=2	 value=(8L, 1L)
debug1 key=8	 value=(2L, 1L)
debug1 key=3	 value=(4L, 1L)
debug1 key=4	 value=(3L, 1L)
debug1 key=3	 value=(5L, 1L)
debug1 key=5	 value=(3L, 1L)
debug1 key=3	 value=(6L, 1L)
debug1 key=6	 value=(3L, 1L)
debug1 key=3	 value=(7L, 1L)
debug1 key=7	 value=(3L, 1L)
debug1 key=3	 value=(8L, 1L)
debug1 key=8	 value=(3L, 1L)
debug1 key=4	 value=(5L, 1L)
debug1 key=5	 value=(4L, 1L)
debug1 key=4	 value=(6L, 1L)
debug1 key=6	 value=(4L, 1L)
debug1 key=4	 value=(7L, 1L)
debug1 

### Step 6: Implement the reduce() function

In [42]:
grouped = pairs.groupByKey()

In [43]:
debug3 = grouped.collect()
for t3 in debug3 :
    print "debug3 key={}\t value={}".format( t3[0],  "".join([str(x) for x in t3[1]] )   )

debug2 key=1	 value=(2L, -1L)(3L, -1L)(4L, -1L)(5L, -1L)(6L, -1L)(7L, -1L)(8L, -1L)(3L, 2L)(4L, 2L)(5L, 2L)(7L, 2L)(2L, 3L)(2L, 4L)(6L, 4L)(2L, 5L)(4L, 6L)(2L, 7L)
debug2 key=2	 value=(3L, 1L)(4L, 1L)(5L, 1L)(6L, 1L)(7L, 1L)(8L, 1L)(1L, -1L)(3L, -1L)(4L, -1L)(5L, -1L)(7L, -1L)(1L, 3L)(1L, 4L)(6L, 4L)(1L, 5L)(1L, 7L)
debug2 key=3	 value=(2L, 1L)(4L, 1L)(5L, 1L)(6L, 1L)(7L, 1L)(8L, 1L)(1L, 2L)(4L, 2L)(5L, 2L)(7L, 2L)(1L, -1L)(2L, -1L)
debug2 key=4	 value=(2L, 1L)(3L, 1L)(5L, 1L)(6L, 1L)(7L, 1L)(8L, 1L)(1L, 2L)(3L, 2L)(5L, 2L)(7L, 2L)(1L, -1L)(2L, -1L)(6L, -1L)(1L, 6L)
debug2 key=5	 value=(2L, 1L)(3L, 1L)(4L, 1L)(6L, 1L)(7L, 1L)(8L, 1L)(1L, 2L)(3L, 2L)(4L, 2L)(7L, 2L)(1L, -1L)(2L, -1L)
debug2 key=6	 value=(2L, 1L)(3L, 1L)(4L, 1L)(5L, 1L)(7L, 1L)(8L, 1L)(1L, 4L)(2L, 4L)(1L, -1L)(4L, -1L)
debug2 key=7	 value=(2L, 1L)(3L, 1L)(4L, 1L)(5L, 1L)(6L, 1L)(8L, 1L)(1L, 2L)(3L, 2L)(4L, 2L)(5L, 2L)(1L, -1L)(2L, -1L)
debug2 key=8	 value=(2L, 1L)(3L, 1L)(4L, 1L)(5L, 1L)(6L, 1L)(7L, 1L)(1L, -1L)


### Step 7: Generate final output

In [57]:
def buildRecommendations( mutualFriends)  :
    from cStringIO import StringIO
    strIOs = StringIO()
    
    for key in mutualFriends.keys() :
        values = mutualFriends[key]
        if values == None :
            continue
        
        strIOs.write( "%s(%d:%s)," 
                     %(key, len( values ), values)
                     )
        
    return strIOs.getvalue()

In [58]:
def make_recommend( values )  :
    mutualFriends = {}  # HashMap 
    for t2 in values :
        toUser = t2[ 0 ]
        mutualFriend = t2[ 1 ]
        alreadyFriend = (mutualFriend == -1L )
        
        if toUser in mutualFriends :
            if alreadyFriend :
                mutualFriends[  toUser ] = None
            elif mutualFriends[  toUser ] != None :
                mutualFriends[  toUser ].append(  mutualFriend )
        else :
            if alreadyFriend :
                mutualFriends[ toUser ] = None
            else :
                list1 = [ mutualFriend ]
                mutualFriends[ toUser ] = list1
    
    return buildRecommendations( mutualFriends )

In [59]:
recommendations = grouped.mapValues( make_recommend )

In [60]:
debug4 = recommendations.collect()
for t4 in debug4 :
    print "debug4 key={}\t value={}".format( t4[0],  "".join([str(x) for x in t4[1]] )   )

debug4 key=1	 value=
debug4 key=2	 value=6(2:[1L, 4L]),8(1:[1L]),
debug4 key=3	 value=4(2:[1L, 2L]),5(2:[1L, 2L]),6(1:[1L]),7(2:[1L, 2L]),8(1:[1L]),
debug4 key=4	 value=3(2:[1L, 2L]),5(2:[1L, 2L]),7(2:[1L, 2L]),8(1:[1L]),
debug4 key=5	 value=3(2:[1L, 2L]),4(2:[1L, 2L]),6(1:[1L]),7(2:[1L, 2L]),8(1:[1L]),
debug4 key=6	 value=2(2:[1L, 4L]),3(1:[1L]),5(1:[1L]),7(1:[1L]),8(1:[1L]),
debug4 key=7	 value=3(2:[1L, 2L]),4(2:[1L, 2L]),5(2:[1L, 2L]),6(1:[1L]),8(1:[1L]),
debug4 key=8	 value=2(1:[1L]),3(1:[1L]),4(1:[1L]),5(1:[1L]),6(1:[1L]),7(1:[1L]),


## MLLIb 활용한 추천

[MovieLens](https://movielens.org/) 데이테셋을 사용한 영화추천 예제

https://databricks-training.s3.amazonaws.com/movie-recommendation-with-mllib.html


ratings.dat
```
UserID::MovieID::Rating::Timestamp
```

movies.dat
```
MovieID::Title::Genres
```

In [1]:
import sys
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS

In [2]:
def parseRating(line):
    """
    Parses a rating record in MovieLens format userId::movieId::rating::timestamp .
    """
    fields = line.strip().split("::")
    return long(fields[3]) % 10, (int(fields[0]), int(fields[1]), float(fields[2]))

In [3]:
def parseMovie(line):
    """
    Parses a movie record in MovieLens format movieId::movieTitle .
    """
    fields = line.strip().split("::")
    return int(fields[0]), fields[1]

In [4]:
def loadRatings(ratingsFile):
    """
    Load ratings from file.
    """
    if not isfile(ratingsFile):
        print "File %s does not exist." % ratingsFile
        sys.exit(1)
    f = open(ratingsFile, 'r')
    ratings = filter(lambda r: r[2] > 0, [parseRating(line)[1] for line in f])
    f.close()
    if not ratings:
        print "No ratings provided."
        sys.exit(1)
    else:
        return ratings

In [5]:
def computeRmse(model, data, n):
    """
    Compute RMSE (Root Mean Squared Error).
    """
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    predictionsAndRatings = predictions.map(lambda x: ((x[0], x[1]), x[2])) \
      .join(data.map(lambda x: ((x[0], x[1]), x[2]))) \
      .values()
    return sqrt(predictionsAndRatings.map(lambda x: (x[0] - x[1]) ** 2).reduce(add) / float(n))

In [6]:
sc = SparkContext() 
sc

In [35]:
# ratings is an RDD of (last digit of timestamp, (userId, movieId, rating))
ratings = sc.textFile( "sample_movielens_ratings.txt" ).map(parseRating)

In [36]:
ratings.collect()[1:10]

[(2L, (0, 3, 1.0)),
 (2L, (0, 5, 2.0)),
 (2L, (0, 9, 4.0)),
 (0L, (0, 11, 1.0)),
 (1L, (0, 12, 2.0)),
 (2L, (0, 15, 1.0)),
 (3L, (0, 17, 1.0)),
 (4L, (0, 19, 1.0)),
 (5L, (0, 21, 1.0))]

In [37]:
# movies is an RDD of (movieId, movieTitle)
movies = dict(sc.textFile( 'sample_movielens_movies.txt' ).map(parseMovie).collect())

In [38]:
movies[1]

u'Movie 1'

In [39]:
numRatings = ratings.count()
numUsers = ratings.values().map(lambda r: r[0]).distinct().count()
numMovies = ratings.values().map(lambda r: r[1]).distinct().count()

print "Got %d ratings from %d users on %d movies." % (numRatings, numUsers, numMovies)

Got 1501 ratings from 30 users on 100 movies.


In [40]:
# split ratings into train (60%), validation (20%), and test (20%) based on the 
# last digit of the timestamp, add myRatings to train, and cache them

# training, validation, test are all RDDs of (userId, movieId, rating
numPartitions = 1
training = ratings.filter(lambda x: x[0] < 6) \
      .values() \
      .repartition(numPartitions) \
      .cache()

In [41]:
validation = ratings.filter(lambda x: x[0] >= 6 and x[0] < 8) \
      .values() \
      .repartition(numPartitions) \
      .cache()

In [42]:
test = ratings.filter(lambda x: x[0] >= 8).values().cache()

In [43]:
numTraining = training.count()
numValidation = validation.count()
numTest = test.count()

print "Training: %d, validation: %d, test: %d" % (numTraining, numValidation, numTest)

Training: 985, validation: 261, test: 255


In [31]:
# train models and evaluate them on the validation set
ranks = [8, 12]
lambdas = [0.1, 10.0]
numIters = [10, 20]
bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [45]:
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    model = ALS.train(training, rank, numIter, lmbda)
    validationRmse = computeRmse(model, validation, numValidation)
    print "RMSE (validation) = %f for the model trained with " % validationRmse + "rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter)
    if (validationRmse < bestValidationRmse):
        bestModel = model
        bestValidationRmse = validationRmse
        bestRank = rank
        bestLambda = lmbda
        bestNumIter = numIter

testRmse = computeRmse(bestModel, test, numTest)

RMSE (validation) = 1.068924 for the model trained with rank = 8, lambda = 0.1, and numIter = 10.
RMSE (validation) = 1.098995 for the model trained with rank = 8, lambda = 0.1, and numIter = 20.
RMSE (validation) = 2.248030 for the model trained with rank = 8, lambda = 10.0, and numIter = 10.
RMSE (validation) = 2.248030 for the model trained with rank = 8, lambda = 10.0, and numIter = 20.
RMSE (validation) = 1.116556 for the model trained with rank = 12, lambda = 0.1, and numIter = 10.
RMSE (validation) = 1.091578 for the model trained with rank = 12, lambda = 0.1, and numIter = 20.
RMSE (validation) = 2.248030 for the model trained with rank = 12, lambda = 10.0, and numIter = 10.
RMSE (validation) = 2.248030 for the model trained with rank = 12, lambda = 10.0, and numIter = 20.


In [46]:
# evaluate the best model on the test set
print "The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) +  "and numIter = %d, and its RMSE on the test set is %f." % (bestNumIter, testRmse)

The best model was trained with rank = 8 and lambda = 0.1, and numIter = 10, and its RMSE on the test set is 1.178326.


In [47]:
# compare the best model with a naive baseline that always returns the mean rating
meanRating = training.union(validation).map(lambda x: x[2]).mean()
baselineRmse = sqrt(test.map(lambda x: (meanRating - x[2]) ** 2).reduce(add) / numTest)
improvement = (baselineRmse - testRmse) / baselineRmse * 100
print "The best model improves the baseline by %.2f" % (improvement) + "%."

The best model improves the baseline by 4.53%.


In [61]:
# make personalized recommendations
myRatings = loadRatings( "sample_movielens_ratings.txt" )

myRatedMovieIds = set([x[1] for x in myRatings])
#candidates = sc.parallelize([m for m in movies if m not in myRatedMovieIds])
candidates = sc.parallelize( myRatedMovieIds )
predictions = bestModel.predictAll(candidates.map(lambda x: (0, x))).collect()
recommendations = sorted(predictions, key=lambda x: x[2], reverse=True)[:50]

In [62]:
recommendations

[Rating(user=0, product=90, rating=2.7518886987713986),
 Rating(user=0, product=2, rating=2.5086793021415383),
 Rating(user=0, product=62, rating=2.318561756589676),
 Rating(user=0, product=22, rating=2.291471061707517),
 Rating(user=0, product=9, rating=2.2304497395145106),
 Rating(user=0, product=81, rating=2.0605470646351933),
 Rating(user=0, product=77, rating=1.9772180354136948),
 Rating(user=0, product=68, rating=1.9587252381228097),
 Rating(user=0, product=32, rating=1.8888954662190125),
 Rating(user=0, product=12, rating=1.862718519383259),
 Rating(user=0, product=41, rating=1.7039251158017619),
 Rating(user=0, product=82, rating=1.6915624171273431),
 Rating(user=0, product=63, rating=1.6672881334205343),
 Rating(user=0, product=95, rating=1.5965051870418856),
 Rating(user=0, product=57, rating=1.5824705802073824),
 Rating(user=0, product=52, rating=1.5802815549028955),
 Rating(user=0, product=28, rating=1.5232135324981972),
 Rating(user=0, product=27, rating=1.4996649971947846